# Data

In [1]:
import pandas as pd


### API URLs ###
# Namara.io API Key
api_key = 'a144567820fbe7c8b88771e3ab2c25ea9e03dc5bc105dad4ef1e9514fa466fb2'

# TORONTO: SAFETY INDICATORS (2014)
api_url_safety = 'https://api.namara.io/v0/data_sets/b668078d-7fb2-4ab0-9c7b-7c39f744b63c/data/en-0?api_key='

# TORONTO: POPULATION BY AGE (2014)
api_url_pop = 'https://api.namara.io/v0/data_sets/728b9e5a-70c0-4a4f-aa6d-62282aaceae4/data/en-0?api_key='

# TORONTO: TORONTO COMMUNITY HOUSING DATA
api_url_housing = 'https://api.namara.io/v0/data_sets/4ed214fd-7401-4162-a668-30d12e58cd38/data/en-1?api_key='

In [2]:
### Load Data ###
data_housing = pd.read_json(api_url_housing + api_key)
data_safety = pd.read_json(api_url_safety + api_key)
data_pop = pd.read_json(api_url_pop + api_key)

In [332]:
# 1. Decide which features you want to keep from each dataset
# 2. Use data_safety as base, remove unwanted features and add data_pop features
# 3. data_housing needs to be aggregated/group for total residents per neighbourhood

print data_housing.shape
data_housing.head(1)

(250, 20)


,bld_desc,bld_id,bldg_postc,buildform,building,dev_id,dev_name,flr_abv_gr,geometry,latitude,longitude,mrkt_unit,nghdnum,objectid,policedivi,rgi_unit,total_resu,x,y,year_built
0,3-Storey Walk-Up Apt. Bldg.,5631,M9P 1T1,Walkup Apartment Building,Towns & Wkups,111,Humber Acres,3,"{u'type': u'Point', u'coordinates': [-79.51653...",43.692662,-79.516537,2,8,550,23,45,47,303451.384,4838967.587,1962


Important fields to keep:

* nghdnum (Neighbourhood Number)
* total_resu (Total Residence)

Only keep housing data where 'year_built' is greater than 2014 (other datasets are restricted to this year).

In [333]:
print data_safety.shape
data_safety.head(1)

(140, 14)


,arsons,assaults,break_enters,fire_fire_alarms,fire_medical_calls,fire_vehicle_incidents,hazardous_incidents,murders,neighbourhood,num,robberies,sexual_assaults,thefts,vehicle_thefts
0,0,66,46,128,275,79,58,1,Agincourt North,129,34,12,1,16


Important fields to keep: All

Will group some such as, robberies and theft or fire_fire_alarms, fire_medical_calls and fire_vehicle_incidents

In [334]:
print data_pop.shape
data_pop.head(1)

(140, 22)


,neighbourhood,num,pop_0_4,pop_10_14,pop_15_19,pop_20_24,pop_25_29,pop_30_34,pop_35_39,pop_40_44,...,pop_55_59,pop_5_to_9,pop_60_64,pop_65_69,pop_70_74,pop_75_79,pop_80_84,pop_85_and_over,pop_female,pop_male
0,Agincourt North,129,"1,320.00","1,560.00","2,000.00","2,165.00","2,145.00","1,685.00","1,715.00","2,125.00",...,"2,105.00","1,345.00","2,000.00","1,430.00","1,245.00","1,025.00",745,650,15845,14445


Important fields to keep:

* num (Neighbourhood Number)
* pop_female
* pop_male

In [335]:
data_safety.head()

,arsons,assaults,break_enters,fire_fire_alarms,fire_medical_calls,fire_vehicle_incidents,hazardous_incidents,murders,neighbourhood,num,robberies,sexual_assaults,thefts,vehicle_thefts
0,0,66,46,128,275,79,58,1,Agincourt North,129,34,12,1,16
1,0,108,73,168,316,158,70,0,Agincourt South-Malvern West,128,24,20,19,32
2,1,44,24,106,170,54,42,1,Alderwood,20,6,5,7,9
3,3,219,156,770,940,135,243,0,Annex,95,34,28,22,14
4,1,61,65,288,544,144,118,0,Banbury-Don Mills,42,12,8,10,20


In [336]:
# Add population totals and safety numbers to new dataset
data = pd.concat([data_safety, data_pop['pop_female'], data_pop['pop_male']], axis=1)
data.head()

,arsons,assaults,break_enters,fire_fire_alarms,fire_medical_calls,fire_vehicle_incidents,hazardous_incidents,murders,neighbourhood,num,robberies,sexual_assaults,thefts,vehicle_thefts,pop_female,pop_male
0,0,66,46,128,275,79,58,1,Agincourt North,129,34,12,1,16,15845,14445
1,0,108,73,168,316,158,70,0,Agincourt South-Malvern West,128,24,20,19,32,11350,10630
2,1,44,24,106,170,54,42,1,Alderwood,20,6,5,7,9,6130,5765
3,3,219,156,770,940,135,243,0,Annex,95,34,28,22,14,15415,13745
4,1,61,65,288,544,144,118,0,Banbury-Don Mills,42,12,8,10,20,14430,12485


In [337]:
data['num'].describe()

count    140.0000
mean      70.5000
std       40.5586
min        1.0000
25%       35.7500
50%       70.5000
75%      105.2500
max      140.0000
Name: num, dtype: float64

In [338]:
# Group housing totals by neighbourhood number
# Add total neighbourhood number to new dataset

# RGI -> Rent-Geared-Income
data_housing_grp_by_nghdum = data_housing.groupby('nghdnum')['total_resu'].sum()

# Verify numbers
print data_housing.loc[data_housing['nghdnum'] == 5]['total_resu'].sum()
data_housing_grp_by_nghdum.head()

115


nghdnum
1    204
2     74
4     64
5    115
6    360
Name: total_resu, dtype: int64

In [339]:
# Add 'total_resu' to dataset matching up housing 'nghdum' to dataset 'num' value
data_housing_by_nghdum = data_housing_grp_by_nghdum[data['num']].reset_index()
data_housing_by_nghdum.head()

,num,total_resu
0,129,NaN
1,128,NaN
2,20,NaN
3,95,NaN
4,42,NaN


In [340]:
data = pd.concat([data, data_housing_by_nghdum['total_resu'].fillna(0)], axis=1)
data.head(15)

,arsons,assaults,break_enters,fire_fire_alarms,fire_medical_calls,fire_vehicle_incidents,hazardous_incidents,murders,neighbourhood,num,robberies,sexual_assaults,thefts,vehicle_thefts,pop_female,pop_male,total_resu
0,0,66,46,128,275,79,58,1,Agincourt North,129,34,12,1,16,15845,14445,0.0
1,0,108,73,168,316,158,70,0,Agincourt South-Malvern West,128,24,20,19,32,11350,10630,0.0
2,1,44,24,106,170,54,42,1,Alderwood,20,6,5,7,9,6130,5765,0.0
3,3,219,156,770,940,135,243,0,Annex,95,34,28,22,14,15415,13745,0.0
4,1,61,65,288,544,144,118,0,Banbury-Don Mills,42,12,8,10,20,14430,12485,0.0
5,0,43,32,153,310,34,106,0,Bathurst Manor,34,6,11,2,36,8050,7380,0.0
6,3,588,97,"1,428.00","1,454.00",268,189,1,Bay Street Corridor,76,104,71,53,24,10030,9315,0.0
7,0,79,47,201,196,106,72,0,Bayview Village,52,12,14,11,25,9420,8245,0.0
8,1,37,22,99,256,28,33,0,Bayview Woods-Steeles,49,6,3,4,16,7275,6260,0.0
9,1,54,84,256,496,142,140,0,Bedford Park-Nortown,39,12,5,11,42,12470,10720,0.0


In [341]:
# The 
total_pop = data['pop_female'] + data['pop_male']
total_pop.rename('total_pop', inplace=True)
data = pd.concat([data, total_pop], axis=1)
data.head()

,arsons,assaults,break_enters,fire_fire_alarms,fire_medical_calls,fire_vehicle_incidents,hazardous_incidents,murders,neighbourhood,num,robberies,sexual_assaults,thefts,vehicle_thefts,pop_female,pop_male,total_resu,total_pop
0,0,66,46,128,275,79,58,1,Agincourt North,129,34,12,1,16,15845,14445,0.0,30290
1,0,108,73,168,316,158,70,0,Agincourt South-Malvern West,128,24,20,19,32,11350,10630,0.0,21980
2,1,44,24,106,170,54,42,1,Alderwood,20,6,5,7,9,6130,5765,0.0,11895
3,3,219,156,770,940,135,243,0,Annex,95,34,28,22,14,15415,13745,0.0,29160
4,1,61,65,288,544,144,118,0,Banbury-Don Mills,42,12,8,10,20,14430,12485,0.0,26915
